In [1]:
import os
import random
import numpy as np
import joblib
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
from skimage.feature import hog
import cv2
from PIL import Image

In [2]:
def load_image(img_path, target_size=(128, 128)):
    img = Image.open(img_path).convert("RGB")
    img = img.resize(target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

def extract_global_features(images, model):
    global_features = []
    for img in images:
        features = model.predict(img)
        features = features.flatten()
        global_features.append(features)
    return np.array(global_features)

def extract_hog_features(images):
    hog_features = []
    for img in images:
        gray_img = cv2.cvtColor(np.squeeze(img).astype('uint8'), cv2.COLOR_RGB2GRAY)
        features = hog(gray_img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

def load_model(filename):
    return joblib.load(filename)

def predict_image(img_path, model, mobilenet_model, target_size=(128, 128)):
    img = load_image(img_path, target_size=target_size)
    global_feat = extract_global_features([img], mobilenet_model)
    local_feat = extract_hog_features([img])
    combined_feat = np.hstack((global_feat, local_feat))
    prediction = model.predict(combined_feat)
    return 'AI-generated' if prediction[0] == 1 else 'Real'

In [3]:
mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [ ]:
def load_dataset_from_multiple_paths(dataset_paths, target_size=(128, 128), sample_size=3000):
    images = []
    labels = []
    
    for dataset_path in dataset_paths:
        for label in ['real', 'ai']:
            folder_path = os.path.join(dataset_path, label)
            img_files = os.listdir(folder_path)
            sampled_files = random.sample(img_files, min(sample_size, len(img_files)))
            
            for img_name in sampled_files:
                img_path = os.path.join(folder_path, img_name)
                img = load_image(img_path, target_size=target_size)
                images.append(img)
                labels.append(0 if label == 'real' else 1)
                
    return np.array(images), np.array(labels)

# dataset paths
dataset_paths = [r'E:\AI Image Detection\Ai Image Dec\Dataset', r"E:\AI Image Detection\program\datasets\train"]
images, labels = load_dataset_from_multiple_paths(dataset_paths)

# Extract features
global_features = extract_global_features(images, mobilenet_model)
local_features = extract_hog_features(images)

# Combine features
features = np.hstack((global_features, local_features))

# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define individual models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
svm_model = SVC(probability=True, kernel='linear', random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, max_depth=3, n_estimators=50)
lr_model = LogisticRegression(max_iter=1000, random_state=42)

# Create Voting Classifier
ensemble_model = VotingClassifier(estimators=[
    ('rf', rf_model),
    ('svm', svm_model),
    ('gb', gb_model),
    ('xgb', xgb_model),
    ('lr', lr_model)
], voting='soft')

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Evaluate the model
y_pred = ensemble_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", report)

# Save the model
model_filename = 'ai_image_detection_ensemble_model1.joblib'
joblib.dump(ensemble_model, model_filename)
print(f"Ensemble model saved as {model_filename}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━

c:\Users\theka\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:10:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 87.94%
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.87      0.88       721
           1       0.87      0.89      0.88       689

    accuracy                           0.88      1410
   macro avg       0.88      0.88      0.88      1410
weighted avg       0.88      0.88      0.88      1410

Ensemble model saved as ai_image_detection_ensemble_model1.joblib


In [4]:
model_filename = 'ai_image_detection_ensemble_model1.joblib'
loaded_model = load_model(model_filename)

new_image_path = r"C:\Users\theka\OneDrive\Desktop\tree.jpg"
prediction = predict_image(new_image_path, loaded_model, mobilenet_model)
print("Prediction:", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 878ms/step
Prediction: Real


In [5]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
import threading
import cv2
import numpy as np
import time


# Assuming the model loading and prediction function have been defined
# and are named `load_model` and `predict_image` respectively.

# Load the pre-trained model
model_filename = 'ai_image_detection_ensemble_model1.joblib'
loaded_model = load_model(model_filename)

# Initialize Tkinter root
root = tk.Tk()
root.title("AI Generated Image Detection")
root.geometry("800x600")
root.config(bg='black')
root.resizable(False, False)

# Global variables to store the uploaded image path and scanning state
uploaded_image_path = None
scan_effect_running = False

# Upload Image function
def upload_image():
    global uploaded_image_path
    uploaded_image_path = filedialog.askopenfilename(
        filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.webp")]
    )
    if uploaded_image_path:
        show_scanning_page()

# Function to show the initial page with the upload button
def show_initial_page():
    clear_frame()
    
    title = tk.Label(root, text="AI Generated Image Detection", font=("Arial", 24, "bold"), 
                     fg="#39FF14", bg="black")
    title.pack(pady=20)

    instruction = tk.Label(root, text="Please upload the image to check whether it is AI or not",
                           font=("Arial", 14), fg="white", bg="black")
    instruction.pack(pady=10)

    upload_button = tk.Button(root, text="Upload", command=upload_image,
                              font=("Arial", 16), fg="black", bg="#39FF14", width=15)
    upload_button.pack(pady=20)

# Function to show the scanning page with the effect
def show_scanning_page():
    clear_frame()
    global scan_effect_running
    scan_effect_running = True

    # Load the uploaded image and display it in Tkinter
    img = Image.open(uploaded_image_path)
    img = img.resize((400, 400))
    tk_img = ImageTk.PhotoImage(img)

    img_label = tk.Label(root, image=tk_img, bg="black")
    img_label.image = tk_img
    img_label.pack(pady=10)

    # Run the scan effect in a separate thread
    threading.Thread(target=scan_effect, args=(img_label,)).start()

# Function to create a scanning effect using OpenCV, with reduced duration
def scan_effect(img_label):
    global scan_effect_running

    # Load image with OpenCV to add scanning effect
    cv_img = cv2.imread(uploaded_image_path)
    cv_img = cv2.resize(cv_img, (400, 400))

    # Calculate number of iterations to fit within 2 seconds
    num_iterations = 50

    # Loop to move a line across the image
    for i in range(num_iterations):
        if not scan_effect_running:
            break

        # Overlay a horizontal line as a scanning effect
        scanned_img = cv_img.copy()
        line_position = int((400 / num_iterations) * i)
        cv2.line(scanned_img, (0, line_position), (400, line_position), (0, 255, 0), 2)

        # Convert back to PIL format for displaying in Tkinter
        scanned_pil_img = Image.fromarray(cv2.cvtColor(scanned_img, cv2.COLOR_BGR2RGB))
        tk_img = ImageTk.PhotoImage(scanned_pil_img)

        # Update label with the new scanned image
        img_label.config(image=tk_img)
        img_label.image = tk_img
        root.update()

        time.sleep(2 / num_iterations)  # Control speed to fit within 2 seconds

    scan_effect_running = False
    show_result_page()

# Function to show the result page with the prediction
def show_result_page():
    clear_frame()

    # Run the prediction and display the result
    try:
        result_text = "AI-generated" if predict_image(uploaded_image_path, loaded_model, mobilenet_model) == "AI-generated" else "Real"
    except Exception as e:
        messagebox.showerror("Error", f"Prediction failed: {e}")
        show_initial_page()
        return

    # Display the result
    result_label = tk.Label(root, text=f"The image is: {result_text}", 
                            font=("Arial", 18, "bold"), fg="#39FF14", bg="black")
    result_label.pack(pady=20)

    # Display the uploaded image
    img = Image.open(uploaded_image_path)
    img = img.resize((400, 400))
    tk_img = ImageTk.PhotoImage(img)

    img_label = tk.Label(root, image=tk_img, bg="black")
    img_label.image = tk_img
    img_label.pack(pady=10)

    # Add a "Back" button to return to the initial page
    back_button = tk.Button(root, text="Back", command=show_initial_page,
                            font=("Arial", 14), fg="black", bg="#39FF14", width=10)
    back_button.pack(pady=20)

# Utility function to clear the GUI frame for switching pages
def clear_frame():
    for widget in root.winfo_children():
        widget.destroy()

# Start with the initial page
show_initial_page()
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
